# Using Valkey as a Vector Database with OpenAI

This notebook provides an introduction to using Valkey as a vector database with OpenAI embeddings. Valkey is scalable, real-time database that can be used as a vector database when using Valkey Search module. The Search module allows to index and search for vectors in Valkey. This notebook will show you how to use the ValkeySearch module to index and search for vectors created by using the OpenAI API and stored in Redis.

### What is Valkey?

Valkey is an open‑source key–value store that originated as a Redis fork under the Linux Foundation. It maintains full compatibility with Redis protocols and APIs while being developed under transparent, community‑driven governance. Developers choose Valkey because it delivers the same fast, reliable performance as Redis with a strong commitment to open‑source principles.

Valkey supports all standard Redis data types and commands, making it a seamless, drop‑in replacement for existing Redis applications.

### What is Valkey Search?

Valkey‑Search is a high‑performance, open‑source Valkey module that adds native vector similarity search and secondary indexing capabilities to Valkey. It is optimized for AI workloads, delivering single‑digit millisecond latency and the ability to handle billions of vectors with over 99% recall. It supports both Approximate Nearest Neighbor (HNSW) and exact K‑Nearest Neighbors (KNN) search, along with complex filtering over Valkey Hash and Valkey‑JSON data. While its current focus is vector search, Valkey‑Search is evolving toward a full search engine with planned support for full‑text search and broader indexing features.

### Deployment options

There are several ways to deploy Valkey with vector search capabilities. For local development, the quickest method is to use the valkey-bundle which we will use here. valkey-bundle contains a number of Valkey modules that can be used together to create a fast data store and query emgine.

For production use cases, you can deploy Valkey on your own infrastructure, use container orchestration with Kubernetes, or deploy on cloud providers.

Since Valkey maintains Redis compatibility, existing Redis deployment patterns and tools work seamlessly with Valkey.


## Prerequisites

Before we start this project, we need to set up the following:

* Start a Valkey database with Search module
* Install required libraries
    * [valkey-glide](https://github.com/valkey-io/valkey-glide)
* Get your [OpenAI API key](https://platform.openai.com/account/api-keys)

===========================================================

### Start Valkey

For this example, we'll use Docker with valkey-bundle, which contains Search module. We can start docker container as follows:

```bash
$ docker-compose up -d
```

You're ready to go! Next, we'll create our client for communicating with the Valkey database.

## Install Requirements

valkey-glide has a Python client for communicating with Valkey. We'll use this to communicate with our Valkey database.

In [ ]:
! pip install valkey-glide openai wget pandas numpy jupyter

===========================================================
## Prepare your OpenAI API key

The OpenAI API key is used for vectorization of query data.

If you don't have an OpenAI API key, you can get one from [https://platform.openai.com/account/api-keys](https://platform.openai.com/account/api-keys).

Once you get your key, add it to your environment variables as `OPENAI_API_KEY`:

In [1]:
! export OPENAI_API_KEY="your API key"

In [15]:
# Test that your OpenAI API key is correctly set as an environment variable
import os
from openai import OpenAI

# Note: alternatively you can set a temporary env variable like this:
# os.environ["OPENAI_API_KEY"] = 'sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'

if os.getenv("OPENAI_API_KEY") is not None:
    print("OPENAI_API_KEY is ready")
else:
    print("OPENAI_API_KEY environment variable not found")

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

OPENAI_API_KEY is ready


## Load Data

In this section we'll load embedded data that has already been converted into vectors. We'll use this data to create an index in Valkey and then search for similar vectors.

In [4]:
import sys
import os
import numpy as np
import pandas as pd
from typing import List

# Use helper function in nbutils.py to download and read the data
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())
import nbutils

nbutils.download_wikipedia_data()
data = nbutils.read_wikipedia_data()

print(f"Loaded {len(data)} articles with embeddings")
data.head()

File Downloaded
Loaded 25000 articles with embeddings


,id,url,title,text,title_vector,content_vector,vector_id
0,1,https://simple.wikipedia.org/wiki/April,April,April is the fourth month of the year in the J...,"[0.001009464613161981, -0.020700545981526375, ...","[-0.011253940872848034, -0.013491976074874401,...",0
1,2,https://simple.wikipedia.org/wiki/August,August,August (Aug.) is the eighth month of the year ...,"[0.0009286514250561595, 0.000820168002974242, ...","[0.0003609954728744924, 0.007262262050062418, ...",1
2,6,https://simple.wikipedia.org/wiki/Art,Art,Art is a creative activity that expresses imag...,"[0.003393713850528002, 0.0061537534929811954, ...","[-0.004959689453244209, 0.015772193670272827, ...",2
3,8,https://simple.wikipedia.org/wiki/A,A,A or a is the first letter of the English alph...,"[0.0153952119871974, -0.013759135268628597, 0....","[0.024894846603274345, -0.022186409682035446, ...",3
4,9,https://simple.wikipedia.org/wiki/Air,Air,Air refers to the Earth's atmosphere. Air is a...,"[0.02224554680287838, -0.02044147066771984, -0...","[0.021524671465158463, 0.018522677943110466, -...",4


## Connect to Valkey

Now that we have our Valkey database running, we can connect to it using the valkey-glide client. We will use the default host and port for the Valkey database which is `localhost:6379`

In [5]:
import asyncio
from glide import GlideClientConfiguration, NodeAddress, GlideClient

async def connect_to_valkey():
    VALKEY_HOST = "localhost"
    VALKEY_PORT = 6379

    addresses = [NodeAddress(VALKEY_HOST, VALKEY_PORT)]
    config = GlideClientConfiguration(
        addresses,
        request_timeout=60000
    )
    client = await GlideClient.create(config)

    # Test connection
    ping_result = await client.ping()
    print(f"Connected to Valkey: {ping_result}")
    return client

# Connect to Valkey
valkey_client = await connect_to_valkey()

Connected to Valkey: b'PONG'


## Creating a Search Index in Valkey

The following cells show how to specify and create a search index in Valkey. We will:

1. Set constants for defining our index (distance metric, index name, etc.)
2. Define the index schema
3. Create the index in Valkey

In [6]:
from glide import ft, glide_json
from glide_shared.commands.server_modules.ft_options.ft_create_options import (
    DataType,
    DistanceMetricType,
    FtCreateOptions,
    TagField,
    VectorAlgorithm,
    VectorField,
    VectorFieldAttributesHnsw,
    VectorFieldAttributesFlat,
    VectorType,
)
# Constants
VECTOR_DIM = len(data['title_vector'][0])    # length of the vectors
VECTOR_NUMBER = len(data)                    # initial number of vectors
INDEX_NAME = "valkey-embeddings-index"       # name of the search index
PREFIX = "doc"                               # prefix for the document keys
DISTANCE_METRIC = DistanceMetricType.COSINE  # distance metric for vectors (COSINE, IP, L2)

print(f"Vector dimensions: {VECTOR_DIM}")
print(f"Number of vectors: {VECTOR_NUMBER}")

Vector dimensions: 1536
Number of vectors: 25000


In [7]:
# Define fields for each column in the dataset
fields = [
    TagField("$.title", alias="title"),
    TagField("$.url", alias="url"),
    TagField("$.text", alias="text"),
    VectorField(
        name="title_vector",
        algorithm=VectorAlgorithm.FLAT,
        attributes=VectorFieldAttributesFlat(
            dimensions=VECTOR_DIM,
            distance_metric=DISTANCE_METRIC,
            type=VectorType.FLOAT32
        )
    ),
    VectorField(
        name="content_vector",
        algorithm=VectorAlgorithm.FLAT,
        attributes=VectorFieldAttributesFlat(
            dimensions=VECTOR_DIM,
            distance_metric=DISTANCE_METRIC,
            type=VectorType.FLOAT32
        )
    )
]
print("Index schema defined")

Index schema defined


In [8]:
# Check if index exists
async def has_index(index_name: str) -> bool:
    try:
        await ft.info(valkey_client, index_name)
        return True
    except:
        return False

async def create_index(index_name: str):
    try:
        if await has_index(index_name):
            print("Index already exists")
            return
        options = FtCreateOptions(
            data_type=DataType.JSON,
            prefixes=[PREFIX]
        )
        await ft.create(valkey_client, index_name, fields, options)
        print(f"Created index: {index_name}")
    except Exception as e:
        print(f"Error creating index: {e}")
        raise e

await create_index(INDEX_NAME)

Index already exists


## Load Documents into the Index

Now that we have a search index, we can load documents into it. We will use the same documents we used in the previous examples. In Valkey, either the HASH or JSON data types can be used to store documents. We will use the JSON data type in this example. The below cells will show how to load documents into the index.

In [9]:
import json
async def index_documents(prefix: str, documents: pd.DataFrame):
    records = documents.to_dict("records")
    for i, doc in enumerate(records):
        key = f"{prefix}:{str(doc['id'])}"
        await glide_json.set(valkey_client, key, "$", json.dumps(doc))
        if i % 100 == 0:
            print(f"Indexed {i} documents")

In [10]:
# Index all documents in Valkey
await index_documents(PREFIX, data.head(5000))
info_result = await valkey_client.info()
print(f"Loaded documents in Valkey search index: {INDEX_NAME}")
db_size = await valkey_client.dbsize()
print(f"Total keys in database: {db_size}")

Indexed 0 documents
Indexed 100 documents
Indexed 200 documents
Indexed 300 documents
Indexed 400 documents
Indexed 500 documents
Indexed 600 documents
Indexed 700 documents
Indexed 800 documents
Indexed 900 documents
Indexed 1000 documents
Indexed 1100 documents
Indexed 1200 documents
Indexed 1300 documents
Indexed 1400 documents
Indexed 1500 documents
Indexed 1600 documents
Indexed 1700 documents
Indexed 1800 documents
Indexed 1900 documents
Indexed 2000 documents
Indexed 2100 documents
Indexed 2200 documents
Indexed 2300 documents
Indexed 2400 documents
Indexed 2500 documents
Indexed 2600 documents
Indexed 2700 documents
Indexed 2800 documents
Indexed 2900 documents
Indexed 3000 documents
Indexed 3100 documents
Indexed 3200 documents
Indexed 3300 documents
Indexed 3400 documents
Indexed 3500 documents
Indexed 3600 documents
Indexed 3700 documents
Indexed 3800 documents
Indexed 3900 documents
Indexed 4000 documents
Indexed 4100 documents
Indexed 4200 documents
Indexed 4300 documents


## Vector Search Queries with OpenAI Embeddings

Now that we have a search index and documents loaded into Valkey, we can run search queries. The following function runs vector similarity searches using OpenAI embeddings as queries.

In [16]:
from glide_shared.commands.server_modules.ft_options.ft_search_options import (
    FtSearchOptions,
    ReturnField,
)

# Convert bytes or bytearray to a UTF-8 string if possible
def _b2s(x: bytes | bytearray | str) -> str:
    if isinstance(x, (bytes, bytearray)):
        try:
            return x.decode("utf-8")
        except Exception:
            return str(x)
    return x

async def search_valkey(
    user_query: str,
    index_name: str = "valkey-embeddings-index",
    vector_field: str = "title_vector",
    hybrid_fields="*",
    k: int = 20,
    print_results: bool = True,
) -> List[dict]:
    # Creates embedding vector from user query
    response = client.embeddings.create(
        input=user_query,
        model="text-embedding-3-small"
    )
    embedded_query = response.data[0].embedding
    query_bytes = np.array(embedded_query).astype(dtype=np.float32).tobytes()

    return_fields = [
        ReturnField("$.title", alias="title"),
        ReturnField("$.url", alias="url"),
        ReturnField("$.text", alias="text"),
        ReturnField(f"vector_score", alias="score")
    ]

    # Prepare the search query
    vector = "query_vector"
    query = f'{hybrid_fields}=>[KNN {k} @{vector_field} ${vector} AS vector_score]'
    query_options = FtSearchOptions(
        params={vector: query_bytes},
        return_fields=return_fields
    )

    # Perform vector search in Valkey
    results = await ft.search(client=valkey_client, index_name=index_name, query=query, options=query_options)

    parsed_results = []
    if results[0] >= 1 and results[1]:
        for key_bytes, fields in results[1].items():
            doc = {
                "key": _b2s(key_bytes),
                "title": _b2s(fields.get(b"title", b"")),
                "url": _b2s(fields.get(b"url", b"")),
                "text": _b2s(fields.get(b"text", b"")),
                "score": float(fields.get(b"vector_score", 0)) if fields.get(b"vector_score") else None
            }
            parsed_results.append(doc)

            if print_results:
                score_str = f"(Score: {round(1-doc['score'], 3)})" if doc['score'] else "(No score)"
                print(f"{doc['key']}: {doc['title']} {score_str}")
    else:
        print("No results found")

    return parsed_results

In [17]:
# Search for articles about modern art in Europe
results = await search_valkey('modern art in Europe', k=10)

doc:4462: Wolfgang Amadeus Mozart (Score: 0.022)
doc:12372: Taylor Dayne (Score: 0.021)
doc:4639: GNU General Public License (Score: 0.021)
doc:13507: Lionel Richie (Score: 0.02)
doc:10602: Lenny Kravitz (Score: 0.019)
doc:4995: GNU (Score: 0.019)
doc:10862: Grand Theft Auto: San Andreas (Score: 0.019)
doc:13757: MP3 (Score: 0.018)
doc:4708: Cessna (Score: 0.018)
doc:14629: Jordan (Score: 0.018)


In [22]:
# Search using content vectors for more detailed matching
results = await search_valkey(
    'modern art in Europe',
    vector_field='content_vector', 
    k=10
)

doc:14502: Boeing 767 (Score: 0.036)
doc:15095: Sublime (Score: 0.036)
doc:14076: Eagles (band) (Score: 0.036)
doc:12363: Duane Eddy (Score: 0.036)
doc:14392: Charlie Christian (Score: 0.035)
doc:10602: Lenny Kravitz (Score: 0.033)
doc:4708: Cessna (Score: 0.032)
doc:12461: Nat King Cole (Score: 0.032)
doc:4714: Jazz (Score: 0.032)
doc:7443: Elvis Presley (Score: 0.028)


## Hybrid Queries with Valkey

The previous examples showed vector search queries with Valkey. In this section, we will show how to combine vector search with other fields for hybrid search. In the below example, we will combine vector search with full text search.

In [24]:
def create_hybrid_field(field_name: str, value: str) -> str:
    return f'@{field_name}:{{{value}}}'

# Search for Scottish history articles with "Scottish" in the title
results = await search_valkey(
    "modern art in Europe",
    vector_field="title_vector",
    k=5,
    hybrid_fields=create_hybrid_field("title", "Jazz")
)

doc:4714: Jazz (Score: 0.011)


## HNSW Index for Improved Performance

So far we've used the FLAT (brute-force) index. Valkey also supports the HNSW (Hierarchical Navigable Small World) index, which provides faster approximate search for large datasets.

HNSW takes longer to build and uses more memory than FLAT, but provides faster query performance, especially for large datasets.

The following cells demonstrate creating and using an HNSW index in Valkey.

In [25]:
# Redefine fields to use HNSW index
hnsw_fields = [
    TagField("$.title", alias="title"),
    TagField("$.url", alias="url"),
    TagField("$.text", alias="text"),
    VectorField(
        name="title_vector",
        algorithm=VectorAlgorithm.HNSW,
        attributes=VectorFieldAttributesHnsw(
            dimensions=VECTOR_DIM,
            distance_metric=DISTANCE_METRIC,
            type=VectorType.FLOAT32
        )
    ),
    VectorField(
        name="content_vector",
        algorithm=VectorAlgorithm.HNSW,
        attributes=VectorFieldAttributesHnsw(
            dimensions=VECTOR_DIM,
            distance_metric=DISTANCE_METRIC,
            type=VectorType.FLOAT32
        )
    )
]
print("HNSW index schema defined")

HNSW index schema defined


In [29]:
async def create_hnsw_index(index_name: str):
    try:
        if await has_index(index_name):
            print("Index already exists")
            return
        options = FtCreateOptions(DataType.JSON, prefixes=[PREFIX])
        await ft.create(valkey_client, index_name, hnsw_fields, options)
        print(f"Created index: {index_name}")
    except Exception as e:
        print(f"Error creating index: {e}")
        raise e

# Create HNSW index
HNSW_INDEX_NAME = INDEX_NAME + "_HNSW"
await create_hnsw_index(HNSW_INDEX_NAME)

Index already exists


In [27]:
# Index all documents in Valkey
await index_documents(PREFIX, data.head(5000))
info_result = await valkey_client.info()
print(f"Loaded documents in Valkey search index: {HNSW_INDEX_NAME}")
db_size = await valkey_client.dbsize()
print(f"Total keys in database: {db_size}")

Indexed 0 documents
Indexed 100 documents
Indexed 200 documents
Indexed 300 documents
Indexed 400 documents
Indexed 500 documents
Indexed 600 documents
Indexed 700 documents
Indexed 800 documents
Indexed 900 documents
Indexed 1000 documents
Indexed 1100 documents
Indexed 1200 documents
Indexed 1300 documents
Indexed 1400 documents
Indexed 1500 documents
Indexed 1600 documents
Indexed 1700 documents
Indexed 1800 documents
Indexed 1900 documents
Indexed 2000 documents
Indexed 2100 documents
Indexed 2200 documents
Indexed 2300 documents
Indexed 2400 documents
Indexed 2500 documents
Indexed 2600 documents
Indexed 2700 documents
Indexed 2800 documents
Indexed 2900 documents
Indexed 3000 documents
Indexed 3100 documents
Indexed 3200 documents
Indexed 3300 documents
Indexed 3400 documents
Indexed 3500 documents
Indexed 3600 documents
Indexed 3700 documents
Indexed 3800 documents
Indexed 3900 documents
Indexed 4000 documents
Indexed 4100 documents
Indexed 4200 documents
Indexed 4300 documents


In [28]:

# Test HNSW index performance
results = await search_valkey(
    'modern art in Europe',
    index_name=HNSW_INDEX_NAME,
    k=10
)

2026-01-05T21:15:08.108359Z  WARN logger_core: received error - Index: field `title_vector` not exists


RequestError: Index: field `title_vector` not exists

## Conclusion

This notebook demonstrated how to use Valkey as a vector database with OpenAI embeddings. Key takeaways:

1. **Valkey High Performance, Compatibility**: Valkey delivers fast, efficient in‑memory data handling and fully supports the Redis protocol, allowing it to function as a seamless drop‑in replacement for Redis while offering strong native performance.

2. **Vector Search Capabilities**: Combined with Search module, Valkey provides powerful vector similarity search with support for multiple distance metrics.

3. **Hybrid Search**: You can combine vector search with traditional text search for more precise results.

4. **Performance Options**: Choose between FLAT (exact) and HNSW (approximate) indexes based on your accuracy and performance requirements.

5. **Open Source**: As an openly developed project under the Linux Foundation, Valkey provides transparent governance and long‑term community‑driven stewardship.

### Resources

- [Valkey Documentation](https://valkey.io/)
- [Valkey Search Module Documentation](https://valkey.io/topics/search/)
- [Valkey-Glide Documentation](https://github.com/valkey-io/valkey-glide/tree/main)
- [OpenAI Embeddings Guide](https://platform.openai.com/docs/guides/embeddings)